In [166]:
import pandas as pd
import MultiContagion as mc
import igraph
import random
import numpy as np
from matplotlib import pylab as plt
import scipy.stats
import copy
from scipy.sparse import linalg as LA
#import powerlaw
%matplotlib inline

In [219]:
year = 2015

In [220]:
#import the aggregate adjacency matrix
aggregate_am = np.genfromtxt ('csv_files/AM4_all_nodes_aggregateNormOutFlow'+str(year)+'.csv', delimiter=",")
df_names = pd.read_csv('csv_files/all_country_name4.csv', header=None)
names = list(df_names[0])
Aggregate_g = igraph.Graph.Weighted_Adjacency(list(aggregate_am))
Aggregate_g.vs["name"] = copy.deepcopy(names)

In [221]:
#Import the adjacency matrix and make graphs of all the layers.
cdis_equity_am = np.genfromtxt ('csv_files/AM4_all_nodesNormOutFlowCDIS-equity'+str(year)+'.csv', delimiter=",")
cdis_equity_g = igraph.Graph.Weighted_Adjacency(list(cdis_equity_am))
cdis_equity_g.vs["name"] = copy.deepcopy(names)

cdis_debt_am = np.genfromtxt ('csv_files/AM4_all_nodesNormOutFlowCDIS-debt'+str(year)+'.csv', delimiter=",")
cdis_debt_g = igraph.Graph.Weighted_Adjacency(list(cdis_debt_am))
cdis_debt_g.vs["name"] = copy.deepcopy(names)

cpis_equity_am = np.genfromtxt ('csv_files/AM4_all_nodesNormOutFlowCPIS-equity'+str(year)+'.csv', delimiter=",")
cpis_equity_g = igraph.Graph.Weighted_Adjacency(list(cpis_equity_am))
cpis_equity_g.vs["name"] = copy.deepcopy(names)

cpis_debt_am = np.genfromtxt ('csv_files/AM4_all_nodesNormOutFlowCPIS-debt'+str(year)+'.csv', delimiter=",")
cpis_debt_g = igraph.Graph.Weighted_Adjacency(list(cpis_debt_am))
cpis_debt_g.vs["name"] = copy.deepcopy(names)

bis_am = np.genfromtxt ('csv_files/AM4_all_nodesNormOutFlowBIS'+str(year)+'.csv', delimiter=",")
bis_g = igraph.Graph.Weighted_Adjacency(list(bis_am))
bis_g.vs["name"] = copy.deepcopy(names)

In [222]:
G_list = [cdis_equity_g, cdis_debt_g, cpis_equity_g, cpis_debt_g, bis_g ]

In [223]:
AM_list = [cdis_equity_am, cdis_debt_am, cpis_equity_am, cpis_debt_am, bis_am ]

In [224]:
#AM_list = [cdis_equity_am, cdis_debt_am, cpis_equity_am, cpis_debt_am, bis_am ]

In [225]:
#G_list2 = [cdis_equity_g,cpis_equity_g, cpis_debt_g, bis_g ]
#AM_list2 = [np.transpose(cdis_equity_am), np.transpose(cpis_equity_am), cpis_debt_am, bis_am ]

In [226]:
In_list = [ np.array(g.strength(mode ="IN", weights= g.es["weight"])) for g in G_list]
Out_list = [ np.array(g.strength(mode ="OUT", weights= g.es["weight"])) for g in G_list]

In [227]:
def make_supra_coup(list_adj, list_vector):
    "Takes a list of adjacency matrix and list of vectors to use as coupling for layers"
    n_layers = len(list_adj)
    dim = list_adj[0].shape[0]
    sup_list = []
    for i in range(n_layers):
        row = []
        for j in range(n_layers):
            if i == j:
                row.append(list_adj[i])
            else:
                row.append(np.diag(list_vector[j]))
        sup_list.append(row)
    return np.bmat(sup_list)

# For real

In [228]:
supra_adj = make_supra_coup(AM_list, In_list)
G_supra = igraph.Graph.Weighted_Adjacency(supra_adj.tolist())
complete_multi_pr = G_supra.personalized_pagerank(weights = G_supra.es["weight"])

In [229]:
G_supra = igraph.Graph.Weighted_Adjacency(supra_adj.tolist())

In [230]:
G_supra = igraph.Graph.Weighted_Adjacency(supra_adj.tolist())

In [231]:
G_supra.is_weighted(), G_supra.is_directed()

(True, True)

In [232]:
def save_centrality(y, namefile, namecent , country, cent):
    f = open("centralities/"+namefile+str(y) +".csv", "w")
    f.write("Country,"+namecent+"\n")
    n = len(country)
    for i in range(n):
        f.write(str(country[i]) + "," + str(cent[i])+ "\n")
    f.close()

## PageRank

In [233]:
pr_agg = Aggregate_g.personalized_pagerank(weights = Aggregate_g.es["weight"], damping = 0.85)
node_names = np.array(copy.deepcopy(names))
inds = np.array(pr_agg).argsort()[::-1][:]
sort_names = node_names[inds]
pr_agg_ord = np.array(pr_agg)[inds]

In [234]:
pr_agg_ord[:10]

array([ 0.12137427,  0.1028246 ,  0.07901446,  0.0506868 ,  0.04953681,
        0.04891865,  0.0486846 ,  0.03990467,  0.03590609,  0.02465681])

In [235]:
sort_names[:30]

array(['Netherlands', 'United States', 'Luxembourg', 'Germany', 'Belgium',
       'France', 'United Kingdom', 'Ireland', 'Switzerland',
       'China  P.R.: Hong Kong', 'Bermuda', 'Italy', 'Cayman Islands',
       'Singapore', 'Russian Federation', 'Sweden',
       'Virgin Islands  British', 'Japan', 'Mexico',
       'China  P.R.: Mainland', 'Spain', 'Korea  Republic of', 'Mauritius',
       'Canada', 'Denmark', 'Turkey', 'Hungary', 'United Arab Emirates',
       'Austria', 'India'], 
      dtype='|S36')

In [236]:
save_centrality(year, "Agg_PageRankOutflow2", "PageRankOutlfow2", sort_names, pr_agg_ord)

In [237]:
def get_pagerank_list(G, n_layers = 5, names= names, r = 0.85):
    pr = G.personalized_pagerank(weights = G.es["weight"], damping = r)
    multi_pr = []
    n_nodes = int((len(G.degree()))/5)
    multi_pr = []
    for i in range(n_nodes):
        cent = 0
        for k in range(n_layers):
            cent += pr[i + n_nodes*k] 
        multi_pr.append(cent)
    node_names = np.array(copy.deepcopy(names))
    inds = np.array(multi_pr).argsort()[::-1][:]
    sort_names_multi_pr = node_names[inds]
    multi_sort_pr = np.array(multi_pr)[inds]
    return sort_names_multi_pr, multi_sort_pr

In [238]:
n_pr, num_pr = get_pagerank_list(G_supra, r = 0.85)

In [239]:
n_pr, num_pr = get_pagerank_list(G_supra, r = 0.85)
n_pr[:30]

array(['Netherlands', 'United States', 'Luxembourg', 'Germany', 'Belgium',
       'United Kingdom', 'France', 'Ireland', 'Switzerland',
       'China  P.R.: Hong Kong', 'Bermuda', 'Italy', 'Cayman Islands',
       'Curacao & St. Maarten', 'Singapore', 'Russian Federation',
       'Sweden', 'Virgin Islands  British', 'Japan', 'Mexico',
       'China  P.R.: Mainland', 'Spain', 'Korea  Republic of', 'Mauritius',
       'Canada', 'Denmark', 'Turkey', 'Hungary', 'United Arab Emirates',
       'Austria'], 
      dtype='|S36')

In [240]:
def make_supra_coup2(list_adj, list_vector):
    "Takes a list of adjacency matrix and list of vectors to use as coupling for layers"
    n_layers = len(list_adj)
    dim = list_adj[0].shape[0]
    sup_list = []
    for i in range(n_layers):
        row = []
        for j in range(n_layers):
            if i == j:
                row.append(list_adj[i])
            else:
                row.append(np.diag(list_vector[i]))
        sup_list.append(row)
    return np.bmat(sup_list)

In [241]:
supra_adj2 = make_supra_coup2(AM_list, In_list)
G_supra2 = igraph.Graph.Weighted_Adjacency(supra_adj2.tolist())

In [242]:
n_pr2, num_pr2 = get_pagerank_list(G_supra2)

In [243]:
n_pr2;

In [244]:
n_pr2, num_pr2 = get_pagerank_list(G_supra2, r = 0.85)
n_pr2[:30]

array(['United States', 'Luxembourg', 'United Kingdom', 'Netherlands',
       'France', 'Germany', 'Japan', 'Switzerland',
       'China  P.R.: Hong Kong', 'Italy', 'Belgium', 'Mauritius',
       'Singapore', 'Korea  Republic of', 'Canada', 'Kuwait', 'Sweden',
       'Norway', 'Denmark', 'Ireland', 'Austria', 'Spain', 'Bermuda',
       'South Africa', 'Cayman Islands', 'China  P.R.: Mainland',
       'Saudi Arabia', 'Russian Federation', 'Jersey', 'Guernsey'], 
      dtype='|S36')

In [245]:
save_centrality(year, "MultCoupCashFlow_PageRankOutflow2", "PageRank centralityOutflow2", n_pr, num_pr)

In [246]:
save_centrality(year, "MultCoupRiskFlow_PageRankOutflow2", "PageRank centralityOutflow2", n_pr2, num_pr2)

## Eigenvector

In [247]:
def get_eigenvector_list(G, n_layers = 5, names= names):
    pr = G.eigenvector_centrality(weights = G.es["weight"])
    multi_pr = []
    n_nodes = int((len(G.degree()))/5)
    multi_pr = []
    for i in range(n_nodes):
        cent = 0
        for k in range(n_layers):
            cent += pr[i + n_nodes*k] 
        multi_pr.append(cent)
    node_names = np.array(copy.deepcopy(names))
    inds = np.array(multi_pr).argsort()[::-1][:]
    sort_names_multi_pr = node_names[inds]
    multi_sort_pr = np.array(multi_pr)[inds]
    return sort_names_multi_pr, multi_sort_pr

In [248]:
n_ei, num_ei = get_eigenvector_list(G_supra)
n_ei2, num_ei2 = get_eigenvector_list(G_supra2)

/home/user/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: RuntimeWarning: Weighted directed graph in eigenvector centrality at centrality.c:362
  from ipykernel import kernelapp as app


In [249]:
n_ei2[:20]

array(['Netherlands', 'United States', 'Luxembourg', 'United Kingdom',
       'Belgium', 'Ireland', 'Germany', 'France', 'Switzerland', 'Bermuda',
       'Cayman Islands', 'China  P.R.: Hong Kong', 'Canada', 'Italy',
       'Sweden', 'Japan', 'Singapore', 'Spain', 'Mexico', 'Hungary'], 
      dtype='|S36')

In [250]:
save_centrality(year, "MultCoupCashFlow_EigenvectorOutflow2", "Eigenvector centralityOutflow2", n_ei, num_ei)
save_centrality(year, "MultCoupRiskFlow_EigenvectorOutflow2", "Eigenvector centralityOutflow2", n_ei2, num_ei2)

## Hubs and Auth

In [251]:
def G_list_hub(G_adj_list):
    G_hub = []
    for g in G_adj_list:
        new = np.dot(g, g.transpose())
        G_hub.append(new)   
    return G_hub

In [252]:
def G_list_auth(G_adj_list):
    G_hub = []
    for g in G_adj_list:
        new = np.dot( g.transpose(), g)
        G_hub.append(new)   
    return G_hub

In [253]:
def hub_auth_diag(Coup_list):
    coup = []
    for c in Coup_list:
        coup.append(c*c)
    return coup

In [254]:
G_supra_hub = make_supra_coup(G_list_hub(AM_list), hub_auth_diag(In_list))
G_supra_auth = make_supra_coup(G_list_auth(AM_list), hub_auth_diag(In_list))

In [255]:
G_supra_hub[0].shape

(1, 1065)

In [256]:
def get_hub_list(AM, n_layers = 5, names= names):
    hub_eig = LA.eigs(AM, k=1, which='LM')[1]
    multi_pr = []
    #print hub_eig[0]
    n_nodes = int((AM[0].shape[1])/5.)
    multi_pr = []
    for i in range(n_nodes):
        cent = 0
        for k in range(n_layers):
            
            cent += abs(hub_eig[i + n_nodes*k])
        
        multi_pr.append(cent[0])
    
    node_names = np.array(copy.deepcopy(names))
    inds = np.array(multi_pr).argsort()[::-1][:]
    sort_names_multi_pr = node_names[inds]
    multi_sort_pr = np.array(multi_pr)[inds]
    return sort_names_multi_pr, multi_sort_pr

In [257]:
n_hub, num_hub = get_hub_list(G_supra_hub)
n_auth, num_auth = get_hub_list(G_supra_auth)

In [258]:
n_auth, num_auth = get_hub_list(G_supra_auth)

In [259]:
n_auth[:20]

array(['Netherlands', 'United States', 'Luxembourg', 'United Kingdom',
       'Belgium', 'Ireland', 'Germany', 'Switzerland', 'France', 'Bermuda',
       'China  P.R.: Hong Kong', 'Cayman Islands', 'Canada', 'Sweden',
       'Italy', 'Japan', 'Spain', 'Mexico', 'Singapore', 'Jersey'], 
      dtype='|S36')

In [260]:
n_hub[:20]

array(['Netherlands', 'United States', 'Luxembourg', 'United Kingdom',
       'Germany', 'Ireland', 'Belgium', 'Switzerland', 'France', 'Spain',
       'Brazil', 'Bermuda', 'Italy', 'Canada', 'Australia',
       'China  P.R.: Hong Kong', 'Russian Federation', 'Sweden',
       'Cayman Islands', 'Japan'], 
      dtype='|S36')

In [261]:
save_centrality(year, "MultCoupCashFlow_HubOutFlow2", "Hub Score2", n_hub, num_hub)
save_centrality(year, "MultCoupCashFlow_AuthOutFlow2", "Authority Score2", n_auth, num_auth)

In [262]:
n_auth[:20]

array(['Netherlands', 'United States', 'Luxembourg', 'United Kingdom',
       'Belgium', 'Ireland', 'Germany', 'Switzerland', 'France', 'Bermuda',
       'China  P.R.: Hong Kong', 'Cayman Islands', 'Canada', 'Sweden',
       'Italy', 'Japan', 'Spain', 'Mexico', 'Singapore', 'Jersey'], 
      dtype='|S36')

In [263]:
n_hub[:20]

array(['Netherlands', 'United States', 'Luxembourg', 'United Kingdom',
       'Germany', 'Ireland', 'Belgium', 'Switzerland', 'France', 'Spain',
       'Brazil', 'Bermuda', 'Italy', 'Canada', 'Australia',
       'China  P.R.: Hong Kong', 'Russian Federation', 'Sweden',
       'Cayman Islands', 'Japan'], 
      dtype='|S36')

### other coup

In [264]:
G_supra_hub2 = make_supra_coup2(G_list_hub(AM_list), hub_auth_diag(In_list))
G_supra_auth2 = make_supra_coup2(G_list_auth(AM_list), hub_auth_diag(In_list))

In [265]:
n_hub2, num_hub2 = get_hub_list(G_supra_hub2)
n_auth2, num_auth2 = get_hub_list(G_supra_auth2)

In [266]:
#n_hub2, num_hub2 = get_eigenvector_list(igraph.Graph.Weighted_Adjacency(G_supra_hub2.tolist()))
#n_auth2, num_auth2 = get_eigenvector_list(igraph.Graph.Weighted_Adjacency(G_supra_auth2.tolist()))

In [267]:
n_auth2[:30]

array(['Netherlands', 'Luxembourg', 'United States', 'United Kingdom',
       'Belgium', 'Ireland', 'Switzerland', 'France', 'Germany', 'Bermuda',
       'Cayman Islands', 'Canada', 'Sweden', 'Italy', 'Japan', 'Mexico',
       'Spain', 'China  P.R.: Hong Kong', 'Singapore', 'Jersey', 'Hungary',
       'China  P.R.: Mainland', 'Australia', 'United Arab Emirates',
       'Denmark', 'Russian Federation', 'Austria', 'Norway', 'Isle of Man',
       'Finland'], 
      dtype='|S36')

In [268]:
n_hub2[:30]

array(['United States', 'Germany', 'United Kingdom', 'Netherlands',
       'Ireland', 'Luxembourg', 'Switzerland', 'Spain', 'France',
       'Belgium', 'Brazil', 'Italy', 'Australia', 'Canada', 'Bermuda',
       'Russian Federation', 'Sweden', 'Japan', 'Cayman Islands', 'Mexico',
       'Norway', 'China  P.R.: Mainland', 'Singapore',
       'China  P.R.: Hong Kong', 'United Arab Emirates', 'Hungary',
       'Mauritius', 'Puerto Rico', 'Poland', 'Austria'], 
      dtype='|S36')

In [269]:
save_centrality(year, "MultCoupRiskFlow_HubOutflow2", "Hub ScoreOutflow2", n_hub2, num_hub2)
save_centrality(year, "MultCoupRiskFlow_AuthOutflow2", "Authority ScoreOutflow2", n_auth2, num_auth2)

In [270]:
num_hub2 [:20]

array([ 0.49965235,  0.43597348,  0.39874015,  0.31059957,  0.25707646,
        0.24039591,  0.18793771,  0.18448201,  0.16754419,  0.14409836,
        0.10743365,  0.09119547,  0.0841655 ,  0.07978462,  0.07504338,
        0.06322695,  0.05531144,  0.05095731,  0.05064792,  0.04896846])